This a similar analysis to the other files except the outcome is binarized by median split


In [1]:
# Connects to your Google Drive so you can import files
from google.colab import drive
drive.mount('/content/drive')

import numpy as np # linear algebra
import pandas as pd

file_path = '/content/drive/My Drive/Colab Notebooks/PC_ROST.csv'
data = pd.read_csv(file_path)
data.sample(4)

Mounted at /content/drive


,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC74,PC75,PC76,PC77,PC78,PC79,PC80,PC81,PC82,binary_score
35,3.442008e+00,3.136514e+00,4.856863e+00,4.376461e+00,-3.003421e+00,8.287181e-01,-2.210457e+00,-2.092644e-01,-8.285292e-01,-1.554201e+00,...,1.159258e+00,-3.665815e-01,3.686161e-01,-1.032133e-01,1.748533e-01,8.354316e-02,8.633238e-01,-1.054073e-01,-2.579779e-01,1
58,3.060000e-15,-2.310000e-15,-7.580000e-16,2.310000e-15,1.120000e-15,3.000000e-15,1.040000e-15,1.480000e-16,-9.660000e-16,2.060000e-17,...,5.130000e-17,1.620000e-16,-8.860000e-16,-1.820000e-16,-2.260000e-16,-3.130000e-16,-4.070000e-16,1.250000e-16,-4.570000e-16,0
86,1.446090e+00,4.168924e+00,1.085663e+00,-1.838860e+00,1.727461e-01,-1.262441e+00,-1.111563e-01,1.157753e+00,2.610872e-02,5.433101e-02,...,5.416295e-01,-2.263033e-01,9.219976e-01,5.444519e-01,-3.047266e-01,1.651507e-01,-3.389036e-01,2.327453e-01,3.748617e-01,0
24,-1.675829e-01,3.169853e+00,-3.470330e+00,5.917025e-02,1.357743e+00,2.364894e+00,-1.370064e+00,-7.224251e-01,-5.327952e-01,1.245571e+00,...,1.554142e-01,5.504771e-01,-2.716984e-01,4.440101e-01,-7.952059e-01,5.844987e-02,1.711799e-01,-1.648338e+00,-7.376731e-03,1


In [2]:
from sklearn.model_selection import train_test_split

# Assuming 'data' is your DataFrame and 'binary_score' is the column with the target variable
X = data.drop('binary_score', axis=1)
y = data['binary_score']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)



In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Create a logistic regression model with L1 regularization
logreg = LogisticRegression(penalty='l1', solver='liblinear', random_state=42)
logreg.fit(X_train, y_train)

# Predict and evaluate the model
y_pred = logreg.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.48148148148148145


In [4]:
from sklearn.svm import LinearSVC

# Create a Linear SVM with L1 regularization
svm = LinearSVC(penalty='l1', dual=False, random_state=42)
svm.fit(X_train, y_train)

# Predict and evaluate
y_pred_svm = svm.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred_svm))



Accuracy: 0.4074074074074074


In [5]:
from sklearn.model_selection import cross_val_score

# Example using Logistic Regression
scores = cross_val_score(logreg, X, y, cv=10)  # 10-fold cross-validation
print("Cross-validated scores:", scores)
print("Average score:", scores.mean())



Cross-validated scores: [0.33333333 0.66666667 0.22222222 0.66666667 0.77777778 0.22222222
 0.55555556 0.44444444 0.66666667 0.625     ]
Average score: 0.5180555555555555


In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score


# Initialize the Random Forest classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the model
rf.fit(X_train, y_train)

# Predict on the test set
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]  # probabilities for the positive class

# Calculate accuracy and ROC AUC score
accuracy = accuracy_score(y_test, y_pred_rf)
roc_auc = roc_auc_score(y_test, y_prob_rf)

print("Accuracy:", accuracy)
print("ROC AUC Score:", roc_auc)



Accuracy: 0.5925925925925926
ROC AUC Score: 0.3920454545454546


In [7]:
# Using cross-validation to assess model performance
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='roc_auc')
print("Cross-validated ROC AUC scores:", cv_scores)
print("Mean ROC AUC score:", cv_scores.mean())


Cross-validated ROC AUC scores: [0.30625    0.625      0.46103896 0.37662338 0.45      ]
Mean ROC AUC score: 0.44378246753246753


In [8]:
from sklearn.model_selection import GridSearchCV

# Parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Setup the grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='roc_auc', verbose=1)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Best parameters and score
print("Best parameters:", grid_search.best_params_)
print("Best ROC AUC score:", grid_search.best_score_)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomFor

Best parameters: {'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best ROC AUC score: 0.5195238095238095


In [10]:
importances = rf.feature_importances_
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print(f"{f + 1}. feature {X_train.columns[indices[f]]} ({importances[indices[f]]})")


Feature ranking:
1. feature PC15 (0.039608286938201065)
2. feature PC41 (0.031258802692741464)
3. feature PC79 (0.026478783925603325)
4. feature PC28 (0.025790488064031676)
5. feature PC24 (0.024284738286210097)
6. feature PC8 (0.023136897373624725)
7. feature PC16 (0.022089206908845117)
8. feature PC30 (0.02147365890404608)
9. feature PC61 (0.02102168630316054)
10. feature PC62 (0.020741189757674167)
11. feature PC39 (0.01958358965217268)
12. feature PC25 (0.019576562278095302)
13. feature PC64 (0.019228676224265093)
14. feature PC54 (0.01852517801337378)
15. feature PC1 (0.018171159508588938)
16. feature PC23 (0.018045955642730337)
17. feature PC29 (0.017794391738375027)
18. feature PC13 (0.01640422283651741)
19. feature PC59 (0.01629847669951471)
20. feature PC78 (0.015898868781597388)
21. feature PC11 (0.015762561810716346)
22. feature PC2 (0.015525849940230969)
23. feature PC65 (0.015266062583023854)
24. feature PC72 (0.014638050322245076)
25. feature PC48 (0.014041602761253713)
2